In [99]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

viajes = pd.read_parquet('../data/interim/viajes_final.parquet')
snapshots = pd.read_parquet('../data/interim/snapshots_final.parquet')
estaciones = pd.read_parquet('../data/interim/estaciones_final.parquet')


In [100]:
print(viajes.head())
print(snapshots.head())
print(estaciones.head())

  genero_usuario edad_usuario     bici id_estacion_retiro        fecha_retiro  \
0              F           22  2586485            107-108 2025-03-31 23:34:46   
1              M           50  8152522                384 2025-03-31 23:43:51   
2              M           24  4219945                021 2025-03-31 23:42:36   
3              M           32  2286540            271-272 2025-03-31 23:57:16   
4              M           26  5017868                538 2025-03-31 23:46:23   

  id_estacion_arribo        fecha_arribo  duracion_viaje franja_operativa  
0                181 2025-04-01 00:00:12       25.433333        normal_lv  
1                387 2025-04-01 00:00:16       16.416667        normal_lv  
2                548 2025-04-01 00:00:23       17.783333        normal_lv  
3                450 2025-04-01 00:00:31        3.250000        normal_lv  
4                012 2025-04-01 00:00:31       14.133333        normal_lv  
                              run_ts id_estacion  \
160 2

In [101]:

snapshots['capacity'] = pd.to_numeric(snapshots['capacity'])
snapshots['bikes_avail'] = pd.to_numeric(snapshots['bikes_avail'])
snapshots['docks_avail'] = pd.to_numeric(snapshots['docks_avail'])



In [102]:
# a. Calcular métricas intermedias necesarias a partir de los snapshots
snapshots['capacidad_total_observada'] = snapshots['bikes_avail'] + snapshots['docks_avail']
snapshots['porcentaje_oos'] = np.divide(
    (snapshots['capacity'] - snapshots['capacidad_total_observada']),
    snapshots['capacity']
).clip(lower=0)

# b. Agrupar por estación y franja para obtener los componentes de la fórmula
df_capacidad_efectiva = snapshots.groupby(['id_estacion', 'franja_operativa']).agg(
    mediana_capacidad_observada=('capacidad_total_observada', 'median'),
    promedio_porcentaje_oos=('porcentaje_oos', 'mean')
).reset_index()

# c. Aplicar la fórmula final para obtener la capacidad efectiva
df_capacidad_efectiva['capacidad_efectiva'] = df_capacidad_efectiva['mediana_capacidad_observada'] * (1 - df_capacidad_efectiva['promedio_porcentaje_oos'])

# d. Mostrar el resultado final para este input
print("\nCálculo de 'capacidad_efectiva' completado. Muestra del resultado:")
print(df_capacidad_efectiva[['id_estacion', 'franja_operativa', 'capacidad_efectiva']].head())



Cálculo de 'capacidad_efectiva' completado. Muestra del resultado:
  id_estacion franja_operativa  capacidad_efectiva
0         166        normal_lv           23.845274
1         166          pico_lv           26.158518
2         167        normal_lv           11.281783
3         167          pico_lv           11.402083
4         168        normal_lv           12.744692


In [103]:

# a. Extraer la capacidad única para cada estación desde el dataframe 'snapshots'.
#    Agrupamos por estación y tomamos el valor máximo de capacidad reportado.
#    (Usamos max() para asegurarnos de obtener el valor de diseño, por si hubiera algún dato erróneo).

capacidades_por_estacion = snapshots.groupby('id_estacion')['capacity'].max().reset_index()


# b. Preparar el dataframe geoespacial.
#    1. Tomamos la lista de estaciones y sus coordenadas del dataframe 'estaciones'.
#    2. Unimos (merge) esa información con las capacidades que acabamos de extraer.
estaciones_geo = pd.merge(
    estaciones[['id_estacion', 'latitud', 'longitud']], # Coordenadas desde 'estaciones'
    capacidades_por_estacion,                           # Capacidad desde 'snapshots'
    on='id_estacion',                                   # La columna en común para unir
    how='left'                                          # 'left' para asegurar que no perdemos estaciones
)


# c. Iterar sobre cada estación para calcular sus métricas de ubicación.
#    (El resto del código es igual, pero ahora usa el dataframe 'estaciones_geo' correctamente construido).
resultados_geo = []
total_estaciones = len(estaciones_geo)


for i, estacion_actual in estaciones_geo.iterrows():
    distancias_a_vecinas = []
    for _, estacion_vecina in estaciones_geo.iterrows():
        if estacion_actual['id_estacion'] != estacion_vecina['id_estacion']:
            dist = geodesic(
                (estacion_actual['latitud'], estacion_actual['longitud']), 
                (estacion_vecina['latitud'], estacion_vecina['longitud'])
            ).meters
            distancias_a_vecinas.append({
                'distancia': dist,
                'capacidad_vecina': estacion_vecina['capacity']
            })

    if not distancias_a_vecinas:
        densidad_500m = 0
        proximidad = 0
    else:
        df_distancias = pd.DataFrame(distancias_a_vecinas)
        
        # Calcular 'densidad_vecina_500m'
        densidad_500m = df_distancias[df_distancias['distancia'] < 500]['capacidad_vecina'].sum()
        
        # Calcular 'proximidad_local'
        distancia_minima = df_distancias['distancia'].min()
        proximidad = np.divide(1, distancia_minima, where=distancia_minima != 0)

    resultados_geo.append({
        'id_estacion': estacion_actual['id_estacion'],
        'densidad_vecina_500m': densidad_500m,
        'proximidad_local_input_dea': proximidad
    })

    if (i + 1) % 10 == 0 or (i + 1) == total_estaciones:
        print(f"  -> Procesadas {i + 1} de {total_estaciones} estaciones.")

# d. Crear el dataframe final con los inputs de ubicación.
df_ubicacion = pd.DataFrame(resultados_geo)

# e. Mostrar una muestra del resultado para verificar.
print("\nCálculo de inputs de ubicación completado. Muestra del resultado:")
print(df_ubicacion.head())

  -> Procesadas 10 de 41 estaciones.
  -> Procesadas 20 de 41 estaciones.
  -> Procesadas 30 de 41 estaciones.
  -> Procesadas 40 de 41 estaciones.
  -> Procesadas 41 de 41 estaciones.

Cálculo de inputs de ubicación completado. Muestra del resultado:
  id_estacion  densidad_vecina_500m  proximidad_local_input_dea
0         258                  27.0                    0.004942
1         447                  35.0                    0.003914
2         049                  62.0                    0.004693
3         050                  27.0                    0.004693
4         053                  62.0                    0.007005


In [104]:
# Asumimos que 'df_ubicacion' ya existe y tiene la columna 'densidad_vecina_300m'.

print("--- Creando Input Numérico de Densidad (0, 1, 2) para DEA ---")

# 1. Crear la columna de input y, por defecto, asignar 0 a todas las estaciones.
#    Este será el valor para la categoría 'Cero'.
df_ubicacion['densidad_input_dea'] = 0

# 2. Identificar las estaciones que NO tienen densidad cero.
estaciones_no_cero = df_ubicacion[df_ubicacion['densidad_vecina_500m'] > 0]

# 3. Dividir el RANGO de valores de las estaciones no cero en 2 partes iguales.
#    Usamos pd.cut con bins=2 para crear dos contenedores de igual tamaño en el rango de valores.
#    Asignamos las etiquetas numéricas 1 (Medio) y 2 (Alto).
categorias_1_y_2 = pd.cut(
    estaciones_no_cero['densidad_vecina_500m'],
    bins=2,
    labels=[1, 2]  # Etiqueta 1 para el primer 50% del rango, 2 para el segundo 50%.
)

# 4. Asignar las categorías 1 y 2 a las filas correspondientes en el dataframe original.
df_ubicacion.loc[estaciones_no_cero.index, 'densidad_input_dea'] = categorias_1_y_2

# --- Verificación del Resultado ---
print("\nConteo de estaciones por cada categoría de input:")
print(df_ubicacion['densidad_input_dea'].value_counts().sort_index())

print("\nLímites de densidad para cada categoría:")
# Para la categoría 0, el min y max es 0.
print("Categoría 0: min=0.0, max=0.0")

# Para las categorías 1 y 2, calculamos los límites reales.
rangos = df_ubicacion[df_ubicacion['densidad_input_dea'] > 0].groupby('densidad_input_dea')['densidad_vecina_500m'].agg(['min', 'max'])
print(rangos)

print("\nMuestra del resultado final:")
# Mostramos la columna original y la nueva columna para comparar.
print(df_ubicacion[['densidad_vecina_500m', 'densidad_input_dea']].sample(10))

--- Creando Input Numérico de Densidad (0, 1, 2) para DEA ---

Conteo de estaciones por cada categoría de input:
densidad_input_dea
0     7
1    21
2    13
Name: count, dtype: int64

Límites de densidad para cada categoría:
Categoría 0: min=0.0, max=0.0
                      min    max
densidad_input_dea              
1                    23.0  106.0
2                   113.0  199.0

Muestra del resultado final:
    densidad_vecina_500m  densidad_input_dea
36                 113.0                   2
30                  27.0                   1
40                  48.0                   1
7                    0.0                   0
33                  99.0                   1
35                  88.0                   1
4                   62.0                   1
29                   0.0                   0
3                   27.0                   1
12                   0.0                   0


In [105]:

# 1. Calcular la desviación estándar de 'bikes_avail' para cada estación.
print("Calculando la desviación estándar de la ocupación por estación...")
std_ocupacion = snapshots.groupby('id_estacion')['bikes_avail'].std().reset_index()
std_ocupacion = std_ocupacion.rename(columns={'bikes_avail': 'std_dev_ocupacion'})

# Manejar el caso de estaciones sin variación (NaN), asignándoles 0.
std_ocupacion['std_dev_ocupacion'] = std_ocupacion['std_dev_ocupacion'].fillna(0)
print(" -> Cálculo completado.")

# 2. Encontrar la máxima desviación estándar en todo el sistema.
max_std_dev = std_ocupacion['std_dev_ocupacion'].max()
print(f"La máxima desviación estándar encontrada en el sistema es: {max_std_dev:.2f}")

# 3. Calcular el input para el DEA.
std_ocupacion['capacidad_efectiva_input_dea'] = max_std_dev - std_ocupacion['std_dev_ocupacion']

# 4. Unir este nuevo input al dataframe 'df_ubicacion'.
#    Esta es la línea corregida. Usamos 'df_ubicacion' como el dataframe principal.
df_ubicacion = pd.merge(df_ubicacion, std_ocupacion[['id_estacion', 'capacidad_efectiva_input_dea']], on='id_estacion', how='left')

# Rellenar con un valor alto por si alguna estación no tuviera datos en 'snapshots'.
df_ubicacion['capacidad_efectiva_input_dea'] = df_ubicacion['capacidad_efectiva_input_dea'].fillna(max_std_dev)


# --- Verificación del Resultado ---
print("\nVerificación del input 'capacidad_efectiva_input_dea' en 'df_ubicacion':")
print(df_ubicacion[['id_estacion', 'capacidad_efectiva_input_dea']].sample(10))

print("\nDescripción estadística del nuevo input:")
print(df_ubicacion['capacidad_efectiva_input_dea'].describe())

Calculando la desviación estándar de la ocupación por estación...
 -> Cálculo completado.
La máxima desviación estándar encontrada en el sistema es: 8.35

Verificación del input 'capacidad_efectiva_input_dea' en 'df_ubicacion':
   id_estacion  capacidad_efectiva_input_dea
19         172                      0.000000
14         167                      4.523133
26         177                      2.294189
40         711                      8.354363
0          258                      5.522228
27         178                      5.259983
31         184                      0.583172
30         075                      8.354363
24         286                      1.116153
17         170                      3.484273

Descripción estadística del nuevo input:
count    41.000000
mean      5.119766
std       3.000107
min       0.000000
25%       2.480690
50%       5.259983
75%       8.354363
max       8.354363
Name: capacidad_efectiva_input_dea, dtype: float64


In [114]:
!jupyter nbconvert --to html "002_Analisis inputs.ipynb"

[NbConvertApp] Converting notebook 002_Analisis inputs.ipynb to html
[NbConvertApp] Writing 306698 bytes to 002_Analisis inputs.html
